# Dimensionado de Nginx

Un problema común en servidores Web es que un pico de usuarios lleve la carga del servidor a un punto donde se agote la memoria RAM del servidor y se comience a utilizar la memoria de intercambio o *SWAP*.

Al el SWAP estar basado en almacenamiento de disco duro, es varias órdenes de magnitud más lento que la memoria RAM por que la velocidad de respuesta de los procesos así como el acceso a dispositivos de almacenamiento bajan considerablemente.

Una vez en este estado incluso el conectarnos por SSH al servidor para detener el servicio o realizar reparación de emergencia podría verse seriamente afectado, por lo que la duración de la caída del sitio se maximiza y en ocasiones hay que optar por reiniciar el servidor ya sea físico o virtual.

Lo mejor es optimizar los servidores Web para que utilicen únicamente los recursos disponibles en el servidor y en una situación de sobre carga solamente una cantidad mínima de usuarios se verían afectados y nos permite inclusive crecer la granja de servidor en agregando más servidores Web evitando la interrupción a los usuarios.

En el caso de Nginx la configuración es bastante más sencilla porque no tienen módulos "MPM" sino que solamente cambiamos dos variables en la configuración. Opcionalmente podemos optimizar los "buffers", timeouts y caché del servidor para lograr avances en desempeño.

In [ ]:
sudo apt-get -y install nginx

Vamos ahora a calcular el peso de el proceso de Apache.

**Importante**: Se debe calcular el peso de Apache cuando ha estado sirviendo contenido. El nuestro se encuentra "virgen" por lo que los resultados podrían ser menores.

Para ello vamos a utilizar la herramienta de ``ps(1)`` la cual lista procesos en sistemas Unix.

Para explicar la línea de comando completa, vamos a ir agregando porciones de código al uso básico de la herramienta ``ps(1)``.

Ejecute los siguientes comandos de forma individual, entendiendo la diferencia entre las ejecuciones y los cambios en la línea de comando de cada uno.

In [ ]:
# Forma tradicional de usar ps(1). Útil para listar todos los proceso del equipo en forma de árbol
ps faxw | head -20

In [ ]:
# ps(1) mostrando solo procesos de Apache
ps -C nginx

In [ ]:
# Ahora solicitamos mostrar la columna de RSS o Resident Set Size
ps -o rss -C nginx

# Por comparación, otras métricas disponibles en ps(1). RSS se considera la más precisa para nuestra necesidad.
ps -o size -C nginx
ps -o vsz -C nginx

In [ ]:
# Para manipular el número, queremos eliminar el encabezado de columnas
ps -o rss -C nginx --no-headers

In [ ]:
# Como existen varios procesos de Apache, nótese que tienen diferentes tamaños de memoria.
# Por esta razón vamos a crear un promedio, aprovechando para traducir el valor a megabytes.
ps -o rss -C nginx --no-headers | awk '{total+=$0} END {print total/NR/1024}'

In [ ]:
# Por último, vamos a guardar el valor dentro de una variable, para poder imprimir nuestros cálculos de Workers
httpd_mem=$(ps -o rss -C nginx --no-headers | awk '{total+=$0} END {print total/NR/1024}')

echo $httpd_mem

In [ ]:
# Ahora podemos seguir utilizando el valor dentro de esta receta de Jupyter.

echo $httpd_mem

### Configuración de Nginx
Iniciamos por medir le memoria disponible, en megas, y guardando el valor en una variable.

También vamos a contar la cantidad de procesadores/cores disponibles en el servidor.

**Importante**: En este tutorial vamos a asumir que si existe un componente de base de datos (PostgreSQL, MySQL, MongoDB, etc) se encuentra en un servidor aparte. Si ud corre la base de datos en el servidor, debe definir cuanta memoria va a dedicar a la base de datos, y elminar esta memoria de la memoria total, así como realizar mediciones para verificar que no se sobre pasa de la memoria asignada.

In [ ]:
mem_total=$(grep MemTotal /proc/meminfo | awk '{print $2/1024}')
cpus_total=$(grep -c processor /proc/cpuinfo)

echo $mem_total
echo $cpus_total

**Archivo**: /etc/nginx/nginx.conf

**Cambio**: Ajuste los valores de worker_process y worker_connections según recursos de su servidor.

In [ ]:
echo "worker_processes $cpus_total;
worker_connections $( echo $mem_total / $httpd_mem | bc );"


Puede opcionalmente cambiar los *buffers* que va a utilizar el proceso de Nginx, donde aumentar los valores va a requerir menos acceso a disco, menos lecturas de archivos, y clientes futuros pueden ser servidos con mayor velocidad, pero esto también va a cambiar el tamaño de memoria de Nginx, por lo cual debe recalcular de nuevo los valores de **worker**.

```nginx
client_body_buffer_size 10K;
client_header_buffer_size 1k;
client_max_body_size 8m;
large_client_header_buffers 2 1k;
```